In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import gcsfs

In [3]:
import pandas as pd
import gcsfs

def load_4q_year_gcs(base_path: str, year: int):
    fs = gcsfs.GCSFileSystem()

    df_num = {}
    df_sub = {}
    df_tag = {}

    for q in [1, 2, 3, 4]:
        qname = f"q{q}"

        num_path = f"{base_path}/{year}q{q}/num.parquet"
        sub_path = f"{base_path}/{year}q{q}/sub.parquet"
        tag_path = f"{base_path}/{year}q{q}/tag.parquet"

        df_num[qname] = pd.read_parquet(num_path, filesystem=fs) if fs.exists(num_path) else None
        df_sub[qname] = pd.read_parquet(sub_path, filesystem=fs) if fs.exists(sub_path) else None
        df_tag[qname] = pd.read_parquet(tag_path, filesystem=fs) if fs.exists(tag_path) else None

    return df_num, df_sub, df_tag

In [4]:
BASE = "gs://sec-financials-edgar/sec_edgar_financials/parquet"

df_num, df_sub, df_tag = load_4q_year_gcs(BASE, 2010)

df_num_q1 = df_num["q1"]
df_num_q2 = df_num["q2"]
df_num_q3 = df_num["q3"]
df_num_q4 = df_num["q4"]

df_sub_q1 = df_sub["q1"]
df_sub_q2 = df_sub["q2"]
df_sub_q3 = df_sub["q3"]
df_sub_q4 = df_sub["q4"]

df_tag_q1 = df_tag["q1"]
df_tag_q2 = df_tag["q2"]
df_tag_q3 = df_tag["q3"]
df_tag_q4 = df_tag["q4"]

In [ ]:
df_sub_all = pd.concat(
    [df_sub_q1, df_sub_q2, df_sub_q3, df_sub_q4],
    ignore_index=True
)

df_sub_all = df_sub_all.drop_duplicates(subset=["adsh"])

In [9]:
df_sub_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3903 entries, 0 to 3902
Data columns (total 36 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   adsh        3903 non-null   object 
 1   cik         3903 non-null   int64  
 2   name        3903 non-null   object 
 3   sic         3903 non-null   int64  
 4   countryba   3902 non-null   object 
 5   stprba      3697 non-null   object 
 6   cityba      3903 non-null   object 
 7   zipba       3871 non-null   object 
 8   bas1        3878 non-null   object 
 9   bas2        1395 non-null   object 
 10  baph        3857 non-null   object 
 11  countryma   3735 non-null   object 
 12  stprma      3558 non-null   object 
 13  cityma      3737 non-null   object 
 14  zipma       3732 non-null   object 
 15  mas1        3737 non-null   object 
 16  mas2        1278 non-null   object 
 17  countryinc  3603 non-null   object 
 18  stprinc     3481 non-null   object 
 19  ein         3903 non-null  

In [11]:
df_sub_all["adsh"].nunique()

3903

In [7]:
df_tag_all = pd.concat(
    [df_tag_q1, df_tag_q2, df_tag_q3, df_tag_q4],
    ignore_index=True
)

df_tag_all = df_tag_all.drop_duplicates(subset=["tag", "version"])

In [14]:
df_tag_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50166 entries, 0 to 56969
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tag       50166 non-null  object
 1   version   50166 non-null  object
 2   custom    50166 non-null  int64 
 3   abstract  50166 non-null  int64 
 4   datatype  50166 non-null  object
 5   iord      50166 non-null  object
 6   crdr      44024 non-null  object
 7   tlabel    50159 non-null  object
 8   doc       48899 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.8+ MB


In [18]:
# Check if this combo is fully unique
df_tag_all.duplicated(subset=["tag", "version"]).sum()

np.int64(0)

In [8]:
df_num_all = pd.concat(
    [df_num_q1, df_num_q2, df_num_q3, df_num_q4],
    ignore_index=True
)

df_num_all = df_num_all.drop_duplicates(
    subset=["adsh", "tag", "version", "ddate", "qtrs", "uom", "segments", "coreg"]
)

In [19]:
df_num_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322905 entries, 0 to 1322904
Data columns (total 10 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   adsh      1322905 non-null  object 
 1   tag       1322905 non-null  object 
 2   version   1322905 non-null  object 
 3   ddate     1322905 non-null  int64  
 4   qtrs      1322905 non-null  int64  
 5   uom       1322905 non-null  object 
 6   segments  351244 non-null   object 
 7   coreg     96474 non-null    object 
 8   value     1311459 non-null  float64
 9   footnote  2478 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 100.9+ MB


In [20]:
df_num_all.duplicated(subset=["adsh", "tag", "version", "ddate", "qtrs", "uom", "segments", "coreg"]).sum()

np.int64(0)

In [21]:
# TAG must be unique
assert not df_tag_all.duplicated(["tag", "version"]).any()

# NUM should not lose rows after merge
print("NUM rows:", len(df_num_all))

NUM rows: 1322905


In [22]:
tag_cols = [
    "tag",
    "version",
    "custom",
    "datatype",
    "crdr",
    "iord",
    "tlabel"
]

df_tag = df_tag_all[tag_cols]

In [23]:
df_num_tag = df_num_all.merge(
    df_tag,
    on=["tag", "version"],
    how="left",
    validate="many_to_one"
)

In [24]:
assert len(df_num_tag) == len(df_num_all)

In [25]:
df_num_tag["datatype"].isna().mean() * 100

np.float64(0.0)

In [26]:
# SUB must be unique on adsh
assert not df_sub_all.duplicated(["adsh"]).any()

print("NUM+TAG rows:", len(df_num_tag))
print("SUB rows:", len(df_sub_all))

NUM+TAG rows: 1322905
SUB rows: 3903


In [28]:
sub_cols = [
'adsh',
 'cik',
 'name',
 'sic',
 'afs',
 'wksi',
 'form',
 'period',
 'fy',
 'fp',
 'filed',
'accepted',
 'prevrpt',
]

df_sub = df_sub_all[sub_cols]

In [29]:
df_fact = df_num_tag.merge(
    df_sub,
    on="adsh",
    how="left",
    validate="many_to_one"
)

In [30]:
assert len(df_fact) == len(df_num_tag)

In [31]:
df_fact["cik"].isna().mean() * 100

np.float64(0.0)

In [32]:
# Ensure numeric types
df_fact["cik"] = df_fact["cik"].astype("int64")

df_fact["fy"] = pd.to_numeric(df_fact["fy"], errors="coerce")
df_fact["period"] = pd.to_numeric(df_fact["period"], errors="coerce")

# Standardize form
df_fact["form"] = df_fact["form"].str.strip()

In [33]:
# Standardize strings
df_fact["tag"] = df_fact["tag"].str.strip()
df_fact["version"] = df_fact["version"].str.strip()

# Ensure numeric value is float
df_fact["value"] = pd.to_numeric(df_fact["value"], errors="coerce")

In [34]:
df_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322905 entries, 0 to 1322904
Data columns (total 27 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   adsh      1322905 non-null  object 
 1   tag       1322905 non-null  object 
 2   version   1322905 non-null  object 
 3   ddate     1322905 non-null  int64  
 4   qtrs      1322905 non-null  int64  
 5   uom       1322905 non-null  object 
 6   segments  351244 non-null   object 
 7   coreg     96474 non-null    object 
 8   value     1311459 non-null  float64
 9   footnote  2478 non-null     object 
 10  custom    1322905 non-null  int64  
 11  datatype  1322905 non-null  object 
 12  crdr      1132442 non-null  object 
 13  iord      1322905 non-null  object 
 14  tlabel    1322891 non-null  object 
 15  cik       1322905 non-null  int64  
 16  name      1322905 non-null  object 
 17  sic       1322905 non-null  int64  
 18  afs       1322343 non-null  object 
 19  wksi      1322905 non

In [35]:
df_fact["form"].value_counts()

form
10-Q      955328
10-K      232617
10-Q/A     56102
8-K        25784
20-F       13263
6-K        12155
20-F/A      9897
10-K/A      7449
S-4         3957
6-K/A       2995
S-4/A       1743
8-K/A        618
POS AM       435
40-F         295
40-F/A       267
Name: count, dtype: int64

In [36]:
# Filter the df_fact to only include 10-K and 10-KA
df_fact = df_fact[df_fact["form"].isin(["10-K", "10-K/A"])]

In [41]:
df_fact["form"].value_counts()

form
10-K      232617
10-K/A      7449
Name: count, dtype: int64

In [37]:
df_fact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240066 entries, 0 to 1322879
Data columns (total 27 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   adsh      240066 non-null  object 
 1   tag       240066 non-null  object 
 2   version   240066 non-null  object 
 3   ddate     240066 non-null  int64  
 4   qtrs      240066 non-null  int64  
 5   uom       240066 non-null  object 
 6   segments  72626 non-null   object 
 7   coreg     9058 non-null    object 
 8   value     239103 non-null  float64
 9   footnote  475 non-null     object 
 10  custom    240066 non-null  int64  
 11  datatype  240066 non-null  object 
 12  crdr      202110 non-null  object 
 13  iord      240066 non-null  object 
 14  tlabel    240060 non-null  object 
 15  cik       240066 non-null  int64  
 16  name      240066 non-null  object 
 17  sic       240066 non-null  int64  
 18  afs       240066 non-null  object 
 19  wksi      240066 non-null  int64  
 20  form    

In [43]:
df_fact["cik"].nunique()

542

In [40]:
df_fact["fy"].value_counts().sort_index()

fy
2009.0    174180
2010.0     63026
Name: count, dtype: int64

In [42]:
# count of unique companies in fy 2010
count_2009 = df_fact[df_fact["fy"] == 2009]["cik"].nunique()
count_2010 = df_fact[df_fact["fy"] == 2010]["cik"].nunique()

print(f"Unique companies in FY 2009: {count_2009} and FY 2010: {count_2010}")

Unique companies in FY 2009: 402 and FY 2010: 137


In [ ]:
common_values = set(df_num_2010["version"]) & set(df_num_2010["version"])
len(common_values)

In [ ]:
len(
    pd.merge(
        df_num_2010[["tag", "version"]].drop_duplicates(),
        df_num_2010[["tag", "version"]].drop_duplicates(),
        on=["tag", "version"]
    )
)

In [ ]:
df_num_tag = df_num_2010.merge(
    df_tag_2010,
    on=["tag", "version"],
    how="left",
    validate="many_to_one"
)

In [ ]:
df_num_tag.head()

In [ ]:
df_num_tag.isna().sum().sort_values(ascending=False).head(10)

In [ ]:
sub_cols = [
    "adsh",
    "cik",
    "name",
    "sic",
    "former",
    "changed",
    "afs",
    "wksi",
    "fye",
    "form",
    "period",
    "fy",
    "fp",
    "filed",
    "accepted",
    "prevrpt",
    "detail",
    "instance",
    "nciks",
    "aciks",
]

df_sub_small = df_sub[sub_cols]

In [ ]:
df_fact = df_num_tag.merge(
    df_sub_small,
    on="adsh",
    how="left",
    validate="many_to_one"
)

In [ ]:
len(df_num_tag), len(df_fact)

In [ ]:
df_fact[sub_cols].isna().sum().sort_values(ascending=False)

In [ ]:
df_fact.duplicated(
    subset=[
        "adsh", "tag", "version", "ddate",
        "qtrs", "uom", "segments", "coreg"
    ]
).sum()

In [ ]:
df_fact.head()

In [ ]:
df_fact.info()

In [ ]:
drop_cols = ["instance", "nciks", "aciks", "changed", "former", "doc", "tlabel"]

In [ ]:
df_fact_cleaned = df_fact.drop(columns=drop_cols)

In [ ]:
df_fact_cleaned.head()

In [ ]:
# filter only 10-K forms
df_fact_10k = df_fact_cleaned[df_fact_cleaned["form"] == "10-K"]

In [ ]:
df_fact_10k.info()

In [ ]:
drop_cols2 = ["footnote", "custom","detail","abstract","iord"]
df_fact_10k_cleaned = df_fact_10k.drop(columns=drop_cols2)

In [ ]:
df_fact_10k_cleaned.info()

In [ ]:
pre_cols = ["adsh", "report", "stmt", "tag", "version", "negating", "line",]
df_pre_small = df_pre[pre_cols].copy()

In [ ]:
# plot the distribution of number of facts per filing
facts_per_filing = df_fact_10k_cleaned.groupby("adsh").size()
plt.figure(figsize=(10, 6))
sns.histplot(facts_per_filing, bins=50, kde=False)
plt.title("Distribution of Number of Facts per 10-K Filing")
plt.xlabel("Number of Facts")
plt.ylabel("Number of Filings")
plt.show()

In [ ]:
# plot distribution of unique tags in 10-K filings with filter fy 2010
df_10k_2010 = df_fact_10k_cleaned[df_fact_10k_cleaned["fy"] == 2010]
unique_tags_per_filing = df_10k_2010.groupby("adsh")["tag"].nunique()
plt.figure(figsize=(10, 6))
sns.histplot(unique_tags_per_filing, bins=50, kde=False)
plt.title("Distribution of Unique Tags per 10-K Filing (FY 2010)")
plt.xlabel("Number of Unique Tags")
plt.ylabel("Number of Filings")
plt.show()

In [ ]:
# unique CIK values in 10-K filings
cik_counts = df_fact_10k_cleaned[["cik","tag"]].nunique()
cik_counts

In [ ]:
tag_counts = df_fact_10k_cleaned["tag"].value_counts()
tag_counts

In [ ]:
tag_company_counts = (
    df_fact_10k_cleaned
    .groupby("tag")["cik"]
    .nunique()
    .sort_values(ascending=False)
)

In [ ]:
tag_company_counts.head(40)

In [ ]:
n_companies = df_fact_10k_cleaned["cik"].nunique()

keep_tags = tag_company_counts[
    tag_company_counts >= 0.70 * n_companies
].index

In [ ]:
cum_share = tag_counts.cumsum() / tag_counts.sum()
top_tags = cum_share[cum_share <= 0.90].index  # keep 90% coverage

df_filtered = df_fact_10k_cleaned[df_fact_10k_cleaned["tag"].isin(top_tags)]

In [ ]:
print(f"length of keep_tags: {len(keep_tags)} and tags are {keep_tags}")

In [ ]:
df_filtered["tag"].nunique()

In [ ]:
df_filtered.info()

In [ ]:
df_pre[df_pre["adsh"] == "0000004904-10-000018"]

In [ ]:
df_pre["tag"].value_counts()

In [ ]:
# plot a box plot for the tag value counts
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.boxplot(x=df_pre["tag"].value_counts())
plt.title("Box Plot of Tag Value Counts")
plt.xlabel("Value Counts")
plt.show()

In [ ]:
# count common tags per adsh and plot a box plot
tag_counts_per_adsh = df_pre.groupby('adsh')['tag'].nunique()
tag_counts_per_adsh.head()
plt.figure(figsize=(12, 6))
sns.boxplot(x=tag_counts_per_adsh)
plt.title("Box Plot of Unique Tag Counts per ADSH")
plt.xlabel("Unique Tag Counts")
plt.show()

In [ ]:
# most repeated unique tags across all filings
df_pre['tag'].value_counts().head(20)

In [ ]:
dup_ct = df_pre_small.duplicated(subset=["adsh", "tag","line", "version","plabel"]).sum()
dup_ct

In [ ]:
pre_key = ["adsh", "tag","line", "version"]

In [ ]:
dups = df_pre[df_pre.duplicated(subset=pre_key, keep=False)] \
    .sort_values(pre_key)

dups.info()

In [ ]:
df_pre.info()

In [ ]:
df_sub.info()

In [ ]:
df_sub["form"].value_counts()

In [ ]:
df_sub["cik"].value_counts()

In [ ]:
df_sub[["cik","form"]].value_counts()

In [ ]:
sub = df_sub.copy()

In [ ]:
forms = {"10-K", "10-K/A", "10-Q"}

sub = sub[sub["form"].isin(forms)]

In [ ]:
sub.info()

In [ ]:
sub["form"].value_counts()

In [ ]:
sub["filed_year"] = pd.to_datetime(sub["filed"]).dt.year

In [ ]:
sub = sub.sort_values(["cik", "filed_year", "filed"])

In [ ]:
latest = (
    sub
    .groupby(["cik", "filed_year"], as_index=False)
    .tail(1)
)

In [ ]:
latest["form"].value_counts()

In [ ]:
count_10k_latest = (latest["form"] == "10-K").sum()
count_10q_latest = (latest["form"] == "10-Q").sum()

In [ ]:
count_10k_latest

In [ ]:
count_10q_latest

In [ ]:
count_10ka_latest = (latest["form"] == "10-K/A").sum()
count_10ka_latest

In [ ]:
def has_forms(group):
    forms = set(group["form"])
    return {
        "has_10k": "10-K" in forms,
        "has_10q": "10-Q" in forms,
        "has_10ka": "10-K/A" in forms,
        "latest_form": group.iloc[-1]["form"]
    }

summary = (
    sub
    .groupby(["cik", "filed_year"])
    .apply(has_forms)
    .apply(pd.Series)
    .reset_index()
)

In [ ]:
# 10-K/A is latest AND both 10-K and 10-Q exist
count_10ka_after_both = (
    (summary["latest_form"] == "10-K/A") &
    (summary["has_10k"]) &
    (summary["has_10q"])
).sum()

In [ ]:
count_10ka_after_both

## Exploring SUB

In [ ]:
df_sub1 = pd.read_parquet("gs://sec-financials-edgar/sec_edgar_financials/parquet/2010q1/sub.parquet")

In [ ]:
df_sub2 = pd.read_parquet("gs://sec-financials-edgar/sec_edgar_financials/parquet/2010q2/sub.parquet")

In [ ]:
df_sub3 = pd.read_parquet("gs://sec-financials-edgar/sec_edgar_financials/parquet/2010q3/sub.parquet")

In [ ]:
df_sub4 = pd.read_parquet("gs://sec-financials-edgar/sec_edgar_financials/parquet/2010q4/sub.parquet")

In [ ]:
df_sub5 = pd.read_parquet("gs://sec-financials-edgar/sec_edgar_financials/parquet/2011q1/sub.parquet")

In [ ]:
df_sub_all = pd.concat(
    [df_sub1, df_sub2, df_sub3, df_sub4],
    axis=0,
    ignore_index=True
)

In [ ]:
df_sub_all.info()

In [ ]:
df_sub_all["cik"].value_counts()

In [ ]:
# filter 314808
df_sub_all_filtered = df_sub_all[df_sub_all["cik"] == 314808]

In [ ]:
display(df_sub_all_filtered[["adsh", "cik", "form", "filed", "fy", "fp","period"]])

In [ ]:
df_sub_all["countryba"].value_counts()

In [ ]:
# filter 314808
df_sub_all_us = df_sub_all[df_sub_all["countryba"] == "US"]

In [ ]:
df_sub_all_us["form"].value_counts()

In [ ]:
df_sub_all_us["cik"].nunique()

In [ ]:
(
    df_sub_all_us
    .groupby(["fy", "form"])
    .size()
    .sort_index()
)

In [ ]:
df_sub_all_us["fp"].value_counts()

In [ ]:
df_annual = df_sub_all_us[df_sub_all_us["fp"] == "FY"]

In [ ]:
df_annual["cik"].nunique()

In [ ]:
df_annual["form"].value_counts()

In [ ]:
df_sub_all_us[df_sub_all_us["fy"] == 2010]["cik"].nunique()

In [ ]:
df_sub_all["cik"].nunique()

In [ ]:
df_sub_all["filed"].value_counts()

In [ ]:
df_sub5_filtered = df_sub5[df_sub5["cik"] == 314808]
df_sub5_filtered[["adsh", "cik", "form", "filed", "fy", "fp","period"]]